> 19-12-25 最后一次修改      
## 读取+清洗nan

In [1]:
import numpy as np
import netCDF4 as nc
import pandas as pd
import numpy.ma as ma

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
import xarray as xar

D:\01Setup\ANACONDA\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
D:\01Setup\ANACONDA\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [4]:
file_path='./data/sanjioil.nc'

In [5]:
ds=nc.Dataset(file_path)

In [6]:
ds_xr=xar.open_dataset(file_path)

In [7]:
ds_xr.dims

Frozen(SortedKeysDict({'trajectory': 2958, 'time': 73}))

In [8]:
ds_xr.data_vars

Data variables:
    age_seconds                               (trajectory, time) float32 ...
    interfacial_area                          (trajectory, time) float32 ...
    mass_evaporated                           (trajectory, time) float32 ...
    water_fraction                            (trajectory, time) float32 ...
    y_wind                                    (trajectory, time) float32 ...
    sea_ice_area_fraction                     (trajectory, time) float32 ...
    density                                   (trajectory, time) float32 ...
    oil_film_thickness                        (trajectory, time) float32 ...
    x_sea_water_velocity                      (trajectory, time) float32 ...
    bulltime                                  (trajectory, time) float32 ...
    x_wind                                    (trajectory, time) float32 ...
    age_emulsion_seconds                      (trajectory, time) float32 ...
    sea_surface_wave_stokes_drift_y_velocity  (trajectory, t

In [9]:
ds_xr.coords

Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
  * time        (time) datetime64[ns] 2018-01-14T22:20:00 ... 2018-01-17T22:20:00
    lon         (trajectory, time) float32 ...
    lat         (trajectory, time) float32 ...

获取所有的维度  
* 注意获取维度时，不可以使用下标进行索引，会报错

## 获取`几个指定维度的`的`DataArray`  
如下可知：
    `mass_oil`有2958个值，该物理量有两个维度，分别为`trajectory`,`time`

### 1- 尝试将xarray转为series

## 2 剔除掉nan的数据，只获取有效数据

#### 查看不同时刻的
并通过`where`剔除掉大于`max`以及`nan`的值  
使用xarray的dropna方法，注意与pandas中的方法有所区别

提示错误：
`ValueError: any must be a single dataset dimension`  
查看文档后发现`xarray.Dataset.dropna`尚不支持同时沿多个维度放置  

[参考](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.dropna.html)  

考虑将xarray转换为dateframe，然后再清洗？  
[上网参考](https://stackoverflow.com/questions/52553925/python-xarray-remove-coordinates-with-all-missing-variables)
比较好的解决办法是：
1. 将`DataSet`->`DataFrame`
2. 在`DataFrame`中通过`pandas` `dropna` 即可

In [49]:
xr_temp_x=ds_xr.isel(time=30)['status']
xr_temp_x

<xarray.DataArray 'status' (trajectory: 2958)>
array([-2147483647,           0, -2147483647, ..., -2147483647, -2147483647,
       -2147483647])
Coordinates:
  * trajectory  (trajectory) int32 1 2 3 4 5 6 ... 2953 2954 2955 2956 2957 2958
    time        datetime64[ns] 2018-01-16T04:20:00
    lon         (trajectory) float32 ...
    lat         (trajectory) float32 ...
Attributes:
    valid_range:    [0 1]
    flag_values:    [0 1]
    flag_meanings:  active evaporated

### 步骤：
先使用xarray.where进行筛选(此处有一个问题，[如何使用.where进行多条件筛选]()),  
之后再使用 df进行筛选。  
此种方式可能比较繁琐

In [50]:
# 剔除大于阈值的，并且为nan的
df_finial=xr_merge.where(xr_merge>=0).to_dataframe().dropna(how='any')
df_finial=df_finial[df_finial.status<=0]
df_finial

,status,time,lon,lat
trajectory,,,,
2,0.0,2018-01-16 04:20:00,125.852074,28.504345
4,0.0,2018-01-16 04:20:00,125.837547,28.535669
5,0.0,2018-01-16 04:20:00,125.746597,28.534342
9,0.0,2018-01-16 04:20:00,125.779015,28.548067
10,0.0,2018-01-16 04:20:00,125.941368,28.543129
...,...,...,...,...
1250,0.0,2018-01-16 04:20:00,125.921379,28.369308
1251,0.0,2018-01-16 04:20:00,125.921432,28.369282
1252,0.0,2018-01-16 04:20:00,125.921486,28.369307


In [45]:
df_finial['lon'].mean()

125.8815689086914

In [46]:
df_finial['lat'].mean()

28.48094940185547